Below saves the frequencies and likelihoods of all gapfills for all patients. Originally this only saved non-wildtype but it is informative to keep numbers of all.
This strategy could be generic, although likelihoods should be updated with empirical frequencies when available (i.e. if those probes have been used on normal tissue or cell lines; in this case they have not)

In [1]:
import sys
from gapfill_utilities import utils as gf_utils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
sys.path.append('/home/blattms1/projects/gapfill/4CL_JAK2/likelihood_model/src/')
import utils as lm_utils

In [2]:
def get_wt_likelihood(gapfill,error_rate_dicts,wt_gapfill):
    likelihoods = lm_utils.get_likelihoods_of_true_allele(gapfill, error_rate_dicts)
    if wt_gapfill in likelihoods:
        return likelihoods[wt_gapfill]
    else:
        return None

error_rate_dir = '/home/blattms1/projects/gapfill/4CL_JAK2/likelihood_model/error_rate_dicts_HB'
error_rate_dicts = lm_utils.get_error_rate_dicts(error_rate_dir)

os.system('mkdir unexpected_gapfill_likelihoods')


mkdir: cannot create directory ‘unexpected_gapfill_likelihoods’: File exists


256

In [3]:
lib = '1'


In [4]:
### first get probe_reads to use for the patient

# for BC in ['BC001', 'BC002', 'BC003', 'BC004', 'BC005', 'BC006', 'BC007', 'BC008', 'BC009', 'BC010', 'BC011', 'BC012', 'BC013', 'BC014', 'BC015', 'BC016']:
for BC in ['BC001']:
    adata_path = '/data1/lareauc/users/blattms/gapfill/wta_adata/20250605_MPN_' + lib + '_' + BC + '.h5ad'

    gf_dirs = {}
    gf_dirs[0] = '/data1/lareauc/projects/gapfill/analysis/20250605_MPN_16plex/gf_MPN_16plex_part' + lib + '_1/BC' + str(int(BC.replace('BC',''))) + '_giftwrap/'
    gf_dirs[1] = '/data1/lareauc/projects/gapfill/analysis/20250605_MPN_16plex/gf_MPN_16plex_part' + lib + '_2/BC' + str(int(BC.replace('BC',''))) + '_giftwrap/'

    min_percent_supporting = 0.9
    collapse_across_probes = True
    for key,gf_dir in gf_dirs.items():
        if key == 0:
            probe_reads = gf_utils.get_input_probe_reads(gf_dir, read_threshold = 0, cell_barcode_suffix = '-' + str(key), adata_path = adata_path, min_percent_supporting=min_percent_supporting, collapse_across_probes=collapse_across_probes)
        else:
            probe_reads = pd.concat([probe_reads, gf_utils.get_input_probe_reads(gf_dir, read_threshold = 0, cell_barcode_suffix = '-' + str(key), adata_path = adata_path, min_percent_supporting=min_percent_supporting, collapse_across_probes=collapse_across_probes)], ignore_index=True)

    exclude_keyword = '0bp'
    manifest = gf_utils.get_manifest(gf_dirs[0])
    if 'gapfill_from_transcriptome' not in probe_reads.columns:
        probe_reads = probe_reads.merge(manifest[['gapfill_from_transcriptome','name']], left_on='probe_idx',right_index=True,how='left')
    probe_reads['gapfill'] = probe_reads['gapfill'].fillna('')
    probe_reads['gapfill_from_transcriptome'] = probe_reads['gapfill_from_transcriptome'].fillna('')
    if probe_reads['name'].isna().any():
        raise ValueError('Probes with no name found in probe reads for ' + BC)
    probe_reads = probe_reads.loc[probe_reads['name'].str.contains(exclude_keyword) == False]

    counts_in_library = len(probe_reads)
    gapfill_frequencies = probe_reads.groupby('probe_idx')['gapfill'].value_counts(normalize=True).rename('frequency').reset_index()
    gapfill_counts = probe_reads.groupby('probe_idx')['gapfill'].value_counts().rename('count_of_this_gapfill').reset_index()
    probe_counts = probe_reads.groupby('probe_idx').size().rename('count_of_this_probe').reset_index()

    to_resolve = probe_reads[['probe_idx','name','gapfill','gapfill_from_transcriptome']].drop_duplicates()
    to_resolve['likelihood'] = to_resolve.apply(
        lambda row: get_wt_likelihood(row['gapfill'], error_rate_dicts, row['gapfill_from_transcriptome']), axis=1
    )
    to_resolve = to_resolve.merge(gapfill_frequencies, on = ['probe_idx','gapfill'], how='left')
    to_resolve = to_resolve.merge(gapfill_counts, on = ['probe_idx','gapfill'], how='left')
    to_resolve = to_resolve.merge(probe_counts, on = ['probe_idx'], how='left')

    to_add = manifest.loc[~(manifest.index.isin(to_resolve['probe_idx'])) & (manifest['name'].str.contains(exclude_keyword) == False)].reset_index().rename(columns={'index': 'probe_idx'})
    to_resolve = pd.concat([to_resolve, to_add[['probe_idx','name','gapfill_from_transcriptome']]]).reset_index(drop=True)
    to_resolve[['count_of_this_gapfill','count_of_this_probe']] = to_resolve[['count_of_this_gapfill','count_of_this_probe']].fillna(0)
    to_resolve.to_csv('unexpected_gapfill_likelihoods/unresolved_gapfills_' + BC + '.csv', index=False)

568648 UMIs found
Collapsing UMIs across probes, 568648 UMIs remaining (100.00%)
Filtering probe reads based on read threshold (0) and min percent supporting (0.9), 566830 UMIs remaining (99.68%)
Filtering cells based on min counts (0) and genes (0) in WTA
Filtering probe reads based on cell barcodes in adata, 552672 UMIs remaining (97.19%)
674061 UMIs found
Collapsing UMIs across probes, 674061 UMIs remaining (100.00%)
Filtering probe reads based on read threshold (0) and min percent supporting (0.9), 671947 UMIs remaining (99.69%)
Filtering cells based on min counts (0) and genes (0) in WTA
Filtering probe reads based on cell barcodes in adata, 650774 UMIs remaining (96.55%)
